# Модуль 3. Домашняя работа

In [39]:
#загружаем библиотеки
import pandas as pd
import numpy as np

#=========библиотеки для подбора гиперпараметров============
from scipy.stats import randint as randint
from scipy.stats import uniform

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold
#===========================================================
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score

try:
    from sklearn.model_selection import validation_curve
except ImportError:
    from sklearn.learning_curve import validation_curve

try:
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import StratifiedKFold
    
import statistics

### Загружаем данные

In [40]:
#Выборка для обучения - таргет присутствует
df_train = pd.read_csv('train.csv', sep=',', encoding='utf8')
#Выборка для валидации - таргет отсутствует
df_valid = pd.read_csv('test.csv', sep=',', encoding='utf8')
df_sample = pd.read_csv('sample_submission.csv', sep=',', encoding='utf8')

### Приведем данные к виду, необходимому для обучения

In [41]:
def preproc(df_init):
    df_output= df_init.copy()
    
    # Удалим ненужные признаки
    df_output = df_output.drop(['_id', 'contact','day_of_week'], axis=1)
    
   # Кодируем категориальные признаки
    df_output = pd.get_dummies(df_output, columns=['month','job','default', 'marital','education', 'housing', \
                                                  'loan', 'poutcome'])

    #т.к. при значении данного атрибута больше 8 мало положительных значений, то пусть данные значения = 0 иначе 1
    df_output['campaign'] = df_output['campaign'].map(lambda s: 0 if s>=8 else 1)
    #повторяем для 'previous'
    df_output['previous'] = df_output['previous'].map(lambda s: 0 if s>=4 else 1)
    
    return df_output

In [42]:
df_preproc_train = df_train.pipe(preproc) #получили необходимый для обучения датасет
df_preproc_valid = df_valid.pipe(preproc) #получили датасет для предсказания

In [43]:
# X - МАТРИЦА ПРИЗНАКОВ ДАННЫХ ДЛЯ ОБУЧЕНИЯ.
# Удаляем таргет из фичей объектов, на которых проходим обучение
X=df_preproc_train.drop('target', axis=1) 
# y - ЦЕЛЕВАЯ ПЕРЕМЕННАЯ ОБУЧЕНИЯ. 
# Выделяем таргет для объектов, на которых проводим обучение
y = df_preproc_train['target'] 
##########################################################################

# МАТРИЦА ПРИЗНАКОВ ДАННЫХ ДЛЯ ТЕСТИРОВАНИЯ. 
X_ftest=df_preproc_valid 

## Обучение и выбор лучшей метрики ROC_AUC

In [44]:
RNDState = 99 #сид рандома

### Метод ближайших соседей

In [45]:
# Определим наилучшие гиперпараметры для данного алгоритма

param_grid = {
    'n_neighbors': randint (2, 5),
    'metric': ['euclidean', 'minkowski'],
    'weights': ['uniform','distance'],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
    }

# Будем делать 3 запуска поиска, т.к. при большем количеств система зависает
cv = StratifiedKFold(n_splits=5, random_state=RNDState, shuffle=True)

model_knn = KNeighborsClassifier()

random_search_knn = RandomizedSearchCV(model_knn, param_distributions=param_grid, n_iter=3, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=RNDState)
#fit()
random_search_knn.fit(X, y)


print ("Лучшее значение:{} ".format(random_search_knn.best_score_))
print ("Лучшие гиперпараметры системы: {}".format(random_search_knn.best_estimator_))

Лучшее значение:0.8548927585681999 
Лучшие гиперпараметры системы: KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')


### Дерево принятия решений

In [46]:
# Определим пространство поиска

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 8),
    'min_samples_leaf': randint(5, 10),
    'class_weight': [None, 'balanced']}

# Будем делать 200 запусков поиска
cv = StratifiedKFold(n_splits=5, random_state=RNDState, shuffle=True)

model_dtc = DecisionTreeClassifier(random_state=RNDState)
random_search_dtc = RandomizedSearchCV(model_dtc, param_distributions=param_grid, n_iter=200, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=RNDState)
#fit()
random_search_dtc.fit(X, y)
print ("Лучшее значение:{} ".format(random_search_dtc.best_score_))
print ("Лучшие гиперпараметры системы: {}".format(random_search_dtc.best_estimator_))

Лучшее значение:0.9366018078137105 
Лучшие гиперпараметры системы: DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=6, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=9, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=99,
            splitter='best')


### Логистическая регрессия

In [47]:
# Определим пространство поиска

param_grid = {
    'solver': ['lbfgs', 'sag', 'saga'],
    'class_weight': [None, 'balanced'],
    'multi_class': ['ovr', 'multinomial']
    }

# Будем делать 10 запусков поиска
cv = StratifiedKFold(n_splits=5, random_state=RNDState, shuffle=True)

model_lr = LogisticRegression(random_state=RNDState)
random_search_lr = RandomizedSearchCV(model_lr, param_distributions=param_grid, n_iter=10, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=RNDState)
#fit()
random_search_lr.fit(X, y)
print ("Лучшее значение:{} ".format(random_search_lr.best_score_))
print ("Лучшие гиперпараметры системы: {}".format(random_search_lr.best_estimator_))

Лучшее значение:0.929518502580553 
Лучшие гиперпараметры системы: LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=99,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)


##### Итак, исходя из полученных данных, дерево принятия решений имеет максимальную точность предсказания. Поэтому предсказания на данных test.csv будем делать с помощью данного алгоритма

In [51]:
model_final = DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=6, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=9, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=83,
            splitter='best')
model_final.fit(X, y)
#выполняем предсказания на валидационных данных
predict = model_final.predict(X_ftest)

In [52]:
#создаем список и переводим его в датафрейм
predict_result_list=list(zip(df_sample['_id'], predict))
predict_result = pd.DataFrame(predict_result_list, columns=['_id','target'])


In [53]:
#сохранение в csv
predict_result.to_csv('file.csv', encoding='utf-8', index=False)